<a href="https://colab.research.google.com/github/mheidari98/Movie-Recommender-Systems/blob/main/LightFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

from google.colab import drive
drive.mount('/content/gdrive')

!pip install lightfm

In [1]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k, reciprocal_rank
from lightfm.data import Dataset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from scipy.sparse import csr_matrix 
from scipy.sparse import coo_matrix 
import time
from lightfm.evaluation import auc_score
import pickle
import re
from sklearn.metrics import mean_squared_error,mean_absolute_error


C:\Users\PJ\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
Dataset_path = "./DataSets/"
Movies_metadata = pd.read_csv(Dataset_path+"MoviesInfo.csv")
ratings_2001 = pd.read_csv(Dataset_path+"ratings_2001.csv")
ratings = pd.read_csv( Dataset_path + "ml-latest/ratings.csv")

In [3]:
counts1 = ratings_2001['userId'].value_counts()
ratings_2001 = ratings_2001[ratings_2001['userId'].isin(counts1[counts1 >= 20].index)]
counts = ratings_2001['rating'].value_counts()
ratings_2001 = ratings_2001[ratings['rating'].isin(counts[counts >= 20].index)]
ratings_2001.shape

<ipython-input-3-64dbe356be0e>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ratings_2001 = ratings_2001[ratings['rating'].isin(counts[counts >= 20].index)]


(4949435, 4)

### LightFM 

In [7]:
del ratings_2001['timestamp']
#del ratings_2001['timestamp']


In [8]:
#Generating ratings matrix from ratings table
ratings_matrix = ratings_2001.pivot(index='userId', columns='movieId', values='rating')
userID = ratings_matrix.index
movieId = ratings_matrix.columns
ratings_matrix.fillna(0, inplace = True)
ratings_matrix = ratings_matrix.astype(np.int32)
print(ratings_matrix.shape)
ratings_matrix.head()

(65783, 4898)


movieId,1,2,3,4,5,6,7,8,9,10,...,5717,6184,6425,6460,6474,6531,6677,6910,6918,8739
userId,,,,,,,,,,,,,,,,,,,,,
6,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
8,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18,0,0,0,0,0,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
def _shuffle(uids, iids, data, random_state):

    shuffle_indices = np.arange(len(uids))
    random_state.shuffle(shuffle_indices)

    return (uids[shuffle_indices],
            iids[shuffle_indices],
            data[shuffle_indices])

def random_train_test_split(interactions_df, test_percentage=0.25,random_state=None):

    interactions = csr_matrix(interactions_df.values)
    if random_state is None:
        random_state = np.random.RandomState()

    interactions = interactions.tocoo()

    shape = interactions.shape
    uids, iids, data = (interactions.row,
                        interactions.col,
                        interactions.data)

    uids, iids, data = _shuffle(uids, iids, data, random_state)

    cutoff = int((1.0 - test_percentage) * len(uids))

    train_idx = slice(None, cutoff)
    test_idx = slice(cutoff, None)

    train = coo_matrix((data[train_idx],
                           (uids[train_idx],
                            iids[train_idx])),
                          shape=shape,
                          dtype=interactions.dtype)
    test = coo_matrix((data[test_idx],
                          (uids[test_idx],
                           iids[test_idx])),
                         shape=shape,
                         dtype=interactions.dtype)

    return train, test


In [10]:
train,test = random_train_test_split(ratings_matrix)

### Warp

In [11]:
model01 = LightFM(loss='warp',
                learning_rate=0.90,
                no_components=150,
                user_alpha=0.000005)

In [ ]:
start_time = time.time()

model01 = model01.fit(train,
                  epochs=10,
                  num_threads=16, verbose=True)

print("--- Run time:  {} mins ---".format((time.time() - start_time)/60))

Epoch:  60%|██████    | 6/10 [46:56<34:18, 514.58s/it]  

In [ ]:
train_precision = precision_at_k(model01, train, k=10).mean()
test_precision = precision_at_k(model01, test, k=10).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))


In [ ]:
train_aucscore = auc_score(model01, train).mean()
test_aucscore = auc_score(model01, test).mean()
print('auc_score: train %.2f, test %.2f.' % (train_aucscore, test_aucscore))

In [ ]:
pred01 = model01.predict_rank(test)

In [ ]:
print('MSE:',mean_squared_error(test.toarray(), pred01.toarray()))
print('MAE:',mean_absolute_error(test.toarray(), pred01.toarray()))
print('RMSE:',np.sqrt(mean_squared_error(test.toarray(), pred01.toarray())))

### Bpr

In [18]:
model02 = LightFM(loss='bpr',
                learning_rate=0.90,
                no_components=150,
                user_alpha=0.000005)

In [19]:
model02 = model02.fit(train,
                  epochs=10,
                  num_threads=16, verbose=True)

Epoch: 100%|██████████| 10/10 [35:51<00:00, 215.14s/it]


In [20]:
train_precision = precision_at_k(model02, train, k=10).mean()
test_precision = precision_at_k(model02, test, k=10).mean()
print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.01, test 0.00.


In [21]:
train_aucscore = auc_score(model02, train).mean()
test_aucscore = auc_score(model02, test).mean()
print('auc_score: train %.2f, test %.2f.' % (train_aucscore, test_aucscore))

auc_score: train 0.93, test 0.90.


In [22]:
pred02 = model02.predict_rank(test)

In [23]:
print('MSE:',mean_squared_error(test.toarray(), pred02.toarray()))
print('MAE:',mean_absolute_error(test.toarray(), pred02.toarray()))
print('RMSE:',np.sqrt(mean_squared_error(test.toarray(), pred02.toarray())))

MSE: 6358.306046853931
MAE: 4.077374557785826
RMSE: 79.73898699415444
